In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd /content/drive/'My Drive'/'kaggle'/'LinkingWritingProcess'

/content/drive/My Drive/kaggle/LinkingWritingProcess


In [29]:
!pip install catboost

In [62]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00


In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

import warnings
warnings.filterwarnings('ignore')
import datetime as dt

from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler
import optuna

./data/sample_submission.csv
./data/test_logs.csv
./data/train_logs.csv
./data/train_scores.csv


## データの読み込み

In [31]:
# 訓練データ読み込み
train_df = pd.read_csv("./data/train_logs.csv")
train_scores = pd.read_csv("./data/train_scores.csv")
# 検証データ読み込み
test_df = pd.read_csv("./data/test_logs.csv")

print("train_df： ", train_df.shape)
print("train_scores: ", train_scores.shape)
print("test_df: ", test_df.shape)

train_df：  (8405898, 11)
train_scores:  (2471, 2)
test_df:  (6, 11)


In [32]:
# 訓練データの表示確認
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [33]:
# 検証データの表示確認
test_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


## データ概要

In [34]:
# 各項目のデータ型の確認
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB


In [35]:
# データの統計データを確認
train_df.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count
count,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06
mean,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,1.222964e+03,2.314687e+02
std,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,9.485242e+02,1.759088e+02
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,4.990000e+02,9.600000e+01
50%,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,1.043000e+03,2.000000e+02
75%,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,1.706000e+03,3.270000e+02
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03


In [36]:
# 欠損値の確認(訓練データ)
for col in train_df.columns:
    pct_missing = np.mean(train_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


In [37]:
# 欠損値の確認(検証データ)
for col in test_df.columns:
    pct_missing = np.mean(test_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


## 特徴量の作成
https://www.kaggle.com/code/polyakovstepan/random-forest-atboost-xgb-baseline

In [38]:
def summary_time(df):
    result = df.groupby('id')['action_time'].sum().reset_index()
    result.rename(columns={'action_time': 'summary_time'}, inplace=True)
    return result

def start_pause(df):
    result = df.groupby('id')['down_time'].min().reset_index()
    result.rename(columns={'down_time':'start_pause'}, inplace=True)
    return result

def enter_click(df):
    copy_df = df
    copy_df['enter_click'] = (copy_df['down_event'] == 'Enter')
    copy_df = copy_df.groupby('id')['enter_click'].sum().reset_index()
    return copy_df

def space_click(df):
    copy_df = df
    copy_df['space_click'] = (copy_df['down_event'] == 'Space')
    copy_df = copy_df.groupby('id')['space_click'].sum().reset_index()
    return copy_df

def backspace_click(df):
    copy_df = df
    copy_df['backspace_click'] = (copy_df['down_event'] == 'Backspace')
    copy_df = copy_df.groupby('id')['backspace_click'].sum().reset_index()
    return copy_df

def symbol_length(df):
    result = df.groupby('id')['cursor_position'].max().reset_index()
    result.rename(columns={'cursor_position':'symbol_length'}, inplace=True)
    return result

def text_length(df):
    result = df.groupby('id')['word_count'].max().reset_index()
    return result

def nonproduction_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Nonproduction').mean() * 100).reset_index()
    result.rename(columns={'activity': 'nonproduction_feature'}, inplace=True)
    return result

def input_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Input').mean() * 100).reset_index()
    result.rename(columns={'activity': 'input_feature'}, inplace=True)
    return result

def remove_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Remove/Cut').mean() * 100).reset_index()
    result.rename(columns={'activity': 'remove_feature'}, inplace=True)
    return result

def mean_action_time(df):
    result = df.groupby('id')['action_time'].mean().reset_index()
    result.rename(columns={'action_time':'mean_action_time'}, inplace=True)
    return result

def replace_feature(df):
    result = df[df['activity'] == 'Replace'].groupby('id').size().reset_index(name='replace_feature')
    return result

def text_change_unique(df):
    result = df.groupby('id')['text_change'].nunique().reset_index()
    result.rename(columns={'text_change': 'tch_unique'}, inplace=True)
    return result

def sentence_size_feature(df):
    result = df[(df['text_change'] == '.') & (df['down_event'] != 'Backspace')].groupby('id').size().reset_index(name = 'number_sentence')
    return result

In [39]:
def getDataset(train_df):
    new_df = summary_time(train_df)

    functions = [
        start_pause,enter_click,space_click,backspace_click,symbol_length,text_length,
        nonproduction_feature, input_feature, remove_feature, mean_action_time, replace_feature,
        text_change_unique, sentence_size_feature
    ]

    for func in functions:
        result_df = func(train_df)
        new_df = pd.merge(new_df, result_df, on='id', how='outer')

    return new_df

In [40]:
# 訓練データの特徴量生成
df = getDataset(train_df)
print('train_info: ', df.shape)
df.head()

train_info:  (2471, 15)


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence
0,001519c8,297243,4526,4,357,417,1539,256,4.693000,78.607743,16.308174,116.246774,7.0,17,21.0
1,0022f953,275391,30623,6,391,260,1676,323,10.350448,78.973105,10.594947,112.221271,1.0,12,15.0
2,0042269b,421201,4441,17,552,439,2291,404,4.231141,84.985493,10.614120,101.837766,7.0,19,21.0
3,0059420b,189596,41395,3,243,152,1047,206,6.362468,83.804627,9.704370,121.848329,1.0,10,13.0
4,0075873a,313702,78470,10,324,517,1402,252,2.844725,76.728566,20.426709,123.943896,NaN,9,23.0


In [41]:
# 検証データの特徴量生成
test = getDataset(test_df)
print('test_info: ', test.shape)
test.shape

test_info:  (3, 15)


(3, 15)

## 欠損値消去

In [42]:
# 訓練データの欠損値確認
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 56%
tch_unique - 0%
number_sentence - 0%


In [43]:
# 検証データの欠損値確認
for col in test.columns:
    pct_missing = np.mean(test[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 100%
tch_unique - 0%
number_sentence - 100%


In [44]:
# replace_featureとnumber_sentenceに欠損値が確認される
# この２つの特徴量を消去する
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in  numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        med = test[col].median()
        df[col] = df[col].fillna(med)

imputing missing values for: replace_feature
imputing missing values for: number_sentence


In [45]:
df_numeric = test.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
for col in numeric_cols:
    missing = test[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        test['{}_ismissing'.format(col)] = missing
        med = test[col].median()
        test[col] = test[col].fillna(med)

imputing missing values for: replace_feature
imputing missing values for: number_sentence


In [46]:
print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 17) test_info:  (3, 17)


In [47]:
num_rows = len(df.index)
low_information_cols = []

for col in df.columns:
  cnts = df[col].value_counts(dropna=False)
  top_pct = (cnts/num_rows).iloc[0]

  if top_pct > 0.95:
    low_information_cols.append(col)
    print(' {0}: {1:.5f}%'.format(col, top_pct*100))
    print(cnts)
    print()

 number_sentence_ismissing: 99.95953%
False    2470
True        1
Name: number_sentence_ismissing, dtype: int64



In [48]:
df = df.drop(columns = ['number_sentence_ismissing'], axis=1)
test = test.drop(columns = ['number_sentence_ismissing'], axis=1)

print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 16) test_info:  (3, 16)


In [49]:
df = pd.merge(df, train_scores, on='id', how='outer')
print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 17) test_info:  (3, 16)


In [50]:
# 欠損値に平均をはめる
df['replace_feature'].fillna(df['replace_feature'].mean(), inplace=True)
test['replace_feature'].fillna(test['replace_feature'].mean(), inplace=True)
df['number_sentence'].fillna(df['number_sentence'].mean(), inplace=True)
test['number_sentence'].fillna(test['number_sentence'].mean(), inplace=True)

## Baseline

In [51]:
# 説明変数と目的変数を設定
X = df.drop(columns = ['id', 'score'], axis=1)
y = df.score
X_total = test.drop(columns=['id'], axis=1)

print('train = ', X.shape, 'test = ', X_total.shape, 'target = ', y.shape)

train =  (2471, 15) test =  (3, 15) target =  (2471,)


In [52]:
# モデルの定義
model_1 = CatBoostRegressor(verbose=0)
model_2 = xgb.XGBRegressor()
model_3 = lgb.LGBMRegressor()
model_4 = RandomForestRegressor()
model_5 = HistGradientBoostingRegressor()

# クロスバリデーションを行なって精度の比較
print('Score baseline CatBoostRegressor: ', cross_val_score(model_1, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline XGBRegressor: ', cross_val_score(model_2, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline LGBMRegressor: ', cross_val_score(model_3, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline RandomForestRegressor: ', cross_val_score(model_4, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline HostGradientBoostingRegressor: ', cross_val_score(model_5, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())

Score baseline CatBoostRegressor:  -0.6889205925773207
Score baseline XGBRegressor:  -0.7256322929966512
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2713
[LightGBM] [Info] Number of data points in the train set: 1976, number of used features: 15
[LightGBM] [Info] Start training from score 3.710273
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 1977, number of used features: 15
[LightGBM] [Info] Start training from score 3.709914
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2713
[LightGBM] [Info]

## データ準備

In [55]:
def variance_threshold(df, th):
  var_thres = VarianceThreshold(threshold=th)
  var_thres.fit(df)
  new_cols = var_thres.get_support()
  return df.iloc[:, new_cols]

In [56]:
X_fit = variance_threshold(X, 0.2)
list_name = (X_fit.columns)
X_total_fit = X_total[list_name]

print('Shape test = ', X_total_fit.shape)
print('Shape train = ', X_fit.shape)

Shape test =  (3, 15)
Shape train =  (2471, 15)


In [57]:
X_fit = X_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_total_fit = X_total_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print('Shape train = ', X_fit.shape)
print('Shape test = ', X_total_fit.shape)

Shape train =  (2471, 15)
Shape test =  (3, 15)


In [59]:
# 訓練データの標準化
std_ = StandardScaler()
X_std = std_.fit_transform(X_fit)
X_total_std = std_.fit_transform(X_total_fit)

## Split and Train

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=42)

In [65]:
from sklearn.metrics import mean_squared_error
def objective(trial):

  ### define params grid to search maximum accuracy ###
  n_estimators = trial.suggest_int('n_estimators', 50, 120)
  max_depth = trial.suggest_int('max_depth', 10, 16)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 15, 25)
  criterion = trial.suggest_categorical('criterion', ['friedman_mse', 'absolute_error'])

  ### modeling with suggested params ###
  model = ExtraTreesRegressor(n_estimators = n_estimators,
                                 max_depth = max_depth,
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion,
                                 random_state = 0)

  ### fit ###
  model.fit(X_train, y_train)
  accuracy = mean_squared_error(y_test, model.predict(X_test))
  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)
study.best_value
study.best_trial.params

[I 2023-11-02 11:37:02,925] A new study created in memory with name: no-name-b84e8813-8e57-40a9-8da2-82287553247c
[I 2023-11-02 11:37:07,693] Trial 0 finished with value: 0.4715053672697813 and parameters: {'n_estimators': 92, 'max_depth': 11, 'max_leaf_nodes': 19, 'criterion': 'absolute_error'}. Best is trial 0 with value: 0.4715053672697813.
[I 2023-11-02 11:37:07,788] Trial 1 finished with value: 0.455810713726129 and parameters: {'n_estimators': 55, 'max_depth': 10, 'max_leaf_nodes': 15, 'criterion': 'friedman_mse'}. Best is trial 0 with value: 0.4715053672697813.


{'n_estimators': 92,
 'max_depth': 11,
 'max_leaf_nodes': 19,
 'criterion': 'absolute_error'}